# मिस्टल मोडेलहरूसँग निर्माण गर्दै

## परिचय

यो पाठमा समेटिनेछ:
- विभिन्न मिस्टल मोडेलहरूको अन्वेषण
- प्रत्येक मोडेलका लागि प्रयोगका केसहरू र परिदृश्यहरू बुझ्ने
- कोड उदाहरणहरूले प्रत्येक मोडेलका विशेष विशेषताहरू देखाउँछन्।


## मिस्टल मोडेलहरू

यस पाठमा, हामी ३ फरक मिस्टल मोडेलहरूबारे जान्नेछौं:
**मिस्टल लार्ज**, **मिस्टल स्मल** र **मिस्टल नेमो**।

यी सबै मोडेलहरू Github Model marketplace मा निःशुल्क उपलब्ध छन्। यस नोटबुकमा रहेको कोडले यी मोडेलहरू प्रयोग गरेर कोड चलाउनेछ। यहाँ Github Models प्रयोग गरेर [एआई मोडेलहरूसँग प्रोटोटाइप बनाउने](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) बारे थप जानकारी छ।


## मिस्टल लार्ज २ (२४०७)
मिस्टल लार्ज २ हाल मिस्टलको प्रमुख मोडेल हो र यो उद्यमिक प्रयोगका लागि डिजाइन गरिएको छ।

यो मोडेलले मूल मिस्टल लार्जको तुलनामा निम्न सुधारहरू ल्याएको छ:
-  ठूलो सन्दर्भ विन्डो - १२८k बनाम ३२k
-  गणित र कोडिङ कार्यहरूमा राम्रो प्रदर्शन - ७६.९% औसत शुद्धता बनाम ६०.४%
-  बहुभाषिक प्रदर्शनमा वृद्धि - भाषाहरूमा समावेश छन्: अंग्रेजी, फ्रेन्च, जर्मन, स्पेनिस, इटालियन, पोर्चुगिज, डच, रुसी, चिनियाँ, जापानी, कोरियन, अरबी, र हिन्दी।

यी विशेषतासँगै, मिस्टल लार्ज निम्न कार्यहरूमा उत्कृष्ट छ:
- *रिट्रिभल अग्मेन्टेड जेनेरेसन (RAG)* - ठूलो सन्दर्भ विन्डोका कारण
- *फंक्शन कलिङ* - यो मोडेलमा स्वदेशी फंक्शन कलिङ छ जसले बाह्य उपकरण र API हरूसँग एकीकरण गर्न सकिन्छ। यी कलहरू समानान्तर रूपमा वा एकपछि अर्को अनुक्रमिक रूपमा गर्न सकिन्छ।
- *कोड जेनेरेसन* - यो मोडेल Python, Java, TypeScript र C++ कोड जेनेरेसनमा उत्कृष्ट छ।


यस उदाहरणमा, हामीले Mistral Large 2 प्रयोग गरेर एउटा पाठ कागजातमा RAG ढाँचा चलाएका छौं। प्रश्न कोरियन भाषामा लेखिएको छ र लेखकले कलेज जानु अघि के गरेका थिए भन्ने सोधिएको छ।

यसमा Cohere Embeddings Model प्रयोग गरेर पाठ कागजात र प्रश्न दुवैको embedding बनाइन्छ। यस उदाहरणका लागि, faiss Python प्याकेजलाई भेक्टर स्टोरको रूपमा प्रयोग गरिएको छ।

Mistral मोडेलमा पठाइने prompt मा प्रश्नहरू र प्रश्नसँग मिल्दोजुल्दो retrieved chunk हरू समावेश गरिन्छ। त्यसपछि मोडेलले प्राकृतिक भाषामा उत्तर दिन्छ।


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## मिस्टल सानो
मिस्टल सानो मिस्टल परिवारको अर्को मोडेल हो जुन प्रिमियर/इन्टरप्राइज श्रेणीमा पर्दछ। नामले नै जनाएजस्तै, यो मोडेल सानो भाषा मोडेल (SLM) हो। मिस्टल सानो प्रयोग गर्दा निम्न फाइदाहरू छन्:
- मिस्टल LLM जस्तै मिस्टल ठूलो र NeMo सँग तुलना गर्दा लागत बचत - ८०% मूल्यमा गिरावट
- कम विलम्बता - मिस्टलका LLM हरूसँग तुलना गर्दा छिटो प्रतिक्रिया
- लचिलो - आवश्यक स्रोतहरूमा कम प्रतिबन्धका साथ विभिन्न वातावरणमा प्रयोग गर्न सकिन्छ।

मिस्टल सानो निम्नका लागि उपयुक्त छ:
- पाठमा आधारित कार्यहरू जस्तै संक्षेपण, भावना विश्लेषण र अनुवाद
- लागत प्रभावकारी भएकाले बारम्बार अनुरोध गरिने एप्लिकेसनहरू
- कम विलम्बता भएका कोड कार्यहरू जस्तै समीक्षा र कोड सुझाव


## मिस्टल सानो र मिस्टल ठूलो तुलना गर्दै

मिस्टल सानो र ठूलो बीचको लेटेन्सीको फरक देखाउनका लागि, तलका सेलहरू चलाउनुहोस्।

तपाईंले ३-५ सेकेन्डको प्रतिक्रिया समयको फरक देख्नुहुनेछ। त्यस्तै, एउटै प्रॉम्प्टमा प्रतिक्रिया लम्बाइ र शैलीमा पनि फरक देख्नुहोस्।


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## मिस्टरेल नेमो

यस पाठमा छलफल गरिएका अन्य दुई मोडेलहरूसँग तुलना गर्दा, मिस्टरेल नेमो मात्र निःशुल्क मोडेल हो जसमा Apache2 लाइसेन्स छ।

यो मिस्टरेलको अघिल्लो खुला स्रोत LLM, मिस्टरेल 7B को अपग्रेडको रूपमा हेरिन्छ।

नेमो मोडेलका केही अन्य विशेषताहरू:

- *अझ प्रभावकारी टोकनाइजेसन:* यो मोडेलले सामान्यतया प्रयोग हुने tiktoken भन्दा Tekken टोकनाइजर प्रयोग गर्छ। यसले धेरै भाषाहरू र कोडमा राम्रो प्रदर्शन गर्न मद्दत गर्छ।

- *फाइनट्युनिङ:* आधारभूत मोडेल फाइनट्युनिङका लागि उपलब्ध छ। यसले फाइनट्युनिङ आवश्यक पर्ने प्रयोगका लागि बढी लचिलोपन दिन्छ।

- *नेभेटिभ फंक्शन कलिङ* - मिस्टरेल लार्ज जस्तै, यो मोडेल फंक्शन कलिङमा तालिम दिइएको छ। यसले यसलाई पहिलो खुला स्रोत मोडेलहरूमध्ये एक बनाउँछ जसले यस्तो सुविधा दिएको छ।


## मिस्टरेल नेमो

यस पाठमा छलफल गरिएका अन्य दुई मोडेलहरूसँग तुलना गर्दा, मिस्टरेल नेमो मात्र निःशुल्क मोडेल हो जसमा Apache2 लाइसेन्स छ।

यो मिस्टरेलको अघिल्लो खुला स्रोत LLM, मिस्टरेल 7B को अपग्रेडको रूपमा हेरिन्छ।

नेमो मोडेलका केही अन्य विशेषताहरू:

- *अझ प्रभावकारी टोकनाइजेसन:* यस मोडेलले सामान्यतया प्रयोग हुने tiktoken भन्दा Tekken टोकनाइजर प्रयोग गर्छ। यसले धेरै भाषाहरू र कोडमा राम्रो प्रदर्शन गर्न मद्दत गर्छ।

- *फाइनट्युनिङ:* बेस मोडेल फाइनट्युनिङका लागि उपलब्ध छ। यसले फाइनट्युनिङ आवश्यक पर्ने प्रयोगका लागि बढी लचिलोपन दिन्छ।

- *नेटिभ फंक्शन कलिङ* - मिस्टरेल लार्ज जस्तै, यो मोडेल पनि फंक्शन कलिङमा तालिम दिइएको छ। यसले यसलाई पहिलो खुला स्रोत मोडेलहरूमध्ये एक बनाउँछ जसले यस्तो सुविधा दिएको छ।


### टोकनाइजरहरूको तुलना गर्दै

यस उदाहरणमा, हामी हेर्नेछौं कि Mistral NeMo ले टोकनाइजेसन कसरी गर्छ भन्ने कुरा Mistral Large सँग तुलना गर्दा।

दुवै उदाहरणले एउटै प्रॉम्प्ट लिन्छन् तर तपाईंले देख्नुहुनेछ कि NeMo ले Mistral Large भन्दा कम टोकनहरू फर्काउँछ।


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## सिकाइ यहाँ रोकिँदैन, यात्रा जारी राख्नुहोस्

यो पाठ समाप्त गरेपछि, हाम्रो [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) हेर्नुहोस् र आफ्नो Generative AI ज्ञान अझ बढाउनुहोस्!



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धि हुन सक्छ। मूल भाषा भएको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
